In [ ]:
# HQC estimator below accepts Pytorch tensors (stored in CPU) for both feature matrix X and target vector y

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import check_classification_targets
from sklearn.preprocessing import normalize
# from joblib import Parallel, delayed
import torch
import torch.nn as nn
from torch.multiprocessing import Pool

class HQC(BaseEstimator, ClassifierMixin):
    """The Helstrom Quantum Centroid (HQC) classifier is a quantum-inspired supervised 
    classification approach for data with binary classes (ie. data with 2 classes only).
                         
    Parameters
    ----------
    rescale : int or float, default = 1
        The dataset rescaling factor. A parameter used for rescaling the dataset. 
    n_copies : int, default = 1
        The number of copies to take for each quantum density. This is equivalent to taking 
        the n-fold Kronecker tensor product for each quantum density.
    encoding : str, default = 'amplit'
        The encoding method used to encode vectors into quantum densities. Possible values:
        'amplit', 'stereo'. 'amplit' means using the amplitude encoding method. 'stereo' means 
        using the inverse of the standard stereographic projection encoding method. Default set 
        to 'amplit'.
    class_wgt : str, default = 'equi'
        The class weights assigned to the Quantum Helstrom observable terms. Possible values: 
        'equi', 'weighted'. 'equi' means assigning equal weights of 1/2 (equiprobable) to the
        two classes in the Quantum Helstrom observable. 'weighted' means assigning weights equal 
        to the proportion of the number of rows in each class to the two classes in the Quantum 
        Helstrom observable. Default set to 'equi'.
    n_jobs : int, default = None
        The number of CPU cores used when parallelizing. If -1 all CPUs are used. If 1 is given, 
        no parallel computing code is used at all. For n_jobs below -1, (n_cpus + 1 + n_jobs) 
        are used. Thus for n_jobs = -2, all CPUs but one are used. None is a marker for ‘unset’ 
        that will be interpreted as n_jobs = 1.
    n_splits : int, default = 1
        The number of subset splits performed on the input dataset row-wise and on the number 
        of eigenvalues/eigenvectors of the Quantum Helstrom observable for optimal speed 
        performance. If 1 is given, no splits are used. For optimal speed, recommend using 
        n_splits = int(numpy.ceil(number of CPU cores used/2)). If memory blow-out occurs, 
        reduce n_splits.
    
    Attributes
    ----------
    classes_ : ndarray, shape (2,)
        Sorted binary classes.
    centroid_ : ndarray, shape (2, n_features + 1, n_features + 1)
        Quantum Centroids for class with index 0 and 1 respectively.
    q_hels_obs_ : ndarray, shape (n_features + 1, n_features + 1)
        Quantum Helstrom observable.
    proj_sum_ : tuple, shape (2, n_features + 1, n_features + 1)
        Sum of the projectors of the Quantum Helstrom observable's eigenvectors, which has
        corresponding positive and negative eigenvalues respectively.
    hels_bound_ : float
        Helstrom bound is the upper bound of the probability that one can correctly 
        discriminate whether a quantum density is of which of the two binary quantum density 
        pattern.          
    """
    # Added binary_only tag as required by sklearn check_estimator
    def _more_tags(self):
        return {'binary_only': True}
    
    
    # Initialize model hyperparameters
    def __init__(self, 
                 rescale = 1, 
                 n_copies = 1, 
                 encoding = 'amplit', 
                 class_wgt = 'equi', 
                 n_jobs = None, 
                 n_splits = 1):
        self.rescale = rescale
        self.n_copies = n_copies
        self.encoding = encoding
        self.class_wgt = class_wgt
        self.n_jobs = n_jobs
        self.n_splits = n_splits
        
    
    # Function for kronecker tensor product for PyTorch tensors
    def kronecker(A, B):
        return torch.einsum('ab,cd->acbd', A, B).view(A.size(0)*B.size(0), A.size(1)*B.size(1))
    
    
    # Function for fit
    def fit(self, X, y):
        """Perform HQC classification with the inverse of the standard stereographic 
        projection encoding, with the option to rescale the dataset prior to encoding.
                
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples. An array of int or float.
        y : array-like, shape (n_samples,)
            The training input binary target values. An array of str, int or float.
            
        Returns
        -------
        self : object
            Returns self.
        """
        # Cast tensors X and y into arrays
        X_arr = X.numpy()
        y_arr = y.numpy()
        
        # Check that X and y have correct shape
        # X, y = check_X_y(X, y)
        X_arr, y_arr = check_X_y(X_arr, y_arr)
        
        # Ensure target y is of non-regression type  
        # Added as required by sklearn check_estimator
        # check_classification_targets(y)
        check_classification_targets(y_arr)
            
        # Send tensor y from CPU to GPU, store binary classes and encode y into binary class
        # indexes 0 and 1
        # self.classes_, y_class_index = np.unique(y, return_inverse = True)
        self.classes_, y_class_index = torch.unique(y.cuda(), return_inverse = True)
        
        # Cast X to float to ensure all following calculations below are done in float  
        # rather than integer, and send tensor X from CPU to GPU
        # X = X.astype(float)
        X = torch.DoubleTensor(X).cuda()
        
        # Rescale X
        X = self.rescale*X
        
        # Calculate sum of squares of each row (sample) in X
        # X_sq_sum = (X**2).sum(axis = 1)
        X_sq_sum = (X**2).sum(dim = 1)
        
        # Number of rows in X
        m = X.shape[0]
        
        # Number of columns in X
        n = X.shape[1]
        
        # Calculate X' using amplitude or inverse of the standard stereographic projection 
        # encoding method
        if self.encoding == 'amplit':
            # X_prime = normalize(np.concatenate((X, np.ones(m).reshape(-1, 1)), axis = 1))
            X_prime = nn.functional.normalize(torch.cat([X, torch.ones(m, dtype=torch.float64).reshape(-1, 1).cuda()], 
                                                        dim = 1), p = 2, dim = 1)
        elif self.encoding == 'stereo':
            # X_prime = (1 / (X_sq_sum + 1)).reshape(-1, 1) \
                      # *(np.concatenate((2*X, (X_sq_sum - 1).reshape(-1, 1)), axis = 1))
            X_prime = (1 / (X_sq_sum + 1)).reshape(-1, 1)*(torch.cat((2*X, (X_sq_sum - 1).reshape(-1, 1)), dim = 1))
        else:
            raise ValueError('encoding should be "amplit" or "stereo"')
                        
        # Function to calculate terms in the Quantum Centroids and quantum Helstrom 
        # observable for each class
        def centroid_terms_func(i):
            # Determine rows (samples) in X' belonging to either class
            X_prime_class = X_prime[y_class_index == i]
            
            # Number of rows (samples) in X' belonging to either class
            m_class = X_prime_class.shape[0]
            
            # Split X' belonging to either class into n_splits subsets, row-wise
            # X_prime_class_split = np.array_split(X_prime_class, 
                                                 # indices_or_sections = self.n_splits, 
                                                 # axis = 0)
            X_prime_class_split = torch.chunk(X_prime_class, 
                                              chunks = self.n_splits,
                                              dim = 0)
            
            # Function to calculate sum of quantum densities belonging to each class, 
            # per subset split
            def X_prime_class_split_func(j):
                # Counter for j-th split of X'
                X_prime_class_split_jth = X_prime_class_split[j]
                
                # Number of rows (samples) in j-th split of X'
                m_class_split = X_prime_class_split_jth.shape[0]
            
                # Number of rows/columns in density matrix
                density_nrow_ncol = (n + 1)**self.n_copies
            
                # Initialize arrays density_sum, centroid and q_hels_obs_terms
                # density_sum = np.zeros((density_nrow_ncol, density_nrow_ncol))
                density_sum = torch.zeros([density_nrow_ncol, density_nrow_ncol], dtype = torch.float64).cuda()
                centroid = density_sum              
                q_hels_obs_terms = density_sum
                for k in range(m_class_split):
                    # Encode vectors into quantum densities
                    X_prime_class_split_each_row = X_prime_class_split_jth[k, :]
                    # density_each_row = np.dot(X_prime_class_split_each_row.reshape(-1, 1),
                                              # X_prime_class_split_each_row.reshape(1, -1))
                    density_each_row = torch.matmul(X_prime_class_split_each_row.reshape(-1, 1),
                                                    X_prime_class_split_each_row.reshape(1, -1))
                
                    # Calculate n-fold Kronecker tensor product
                    if self.n_copies == 1:
                        density_each_row = density_each_row
                    else:
                        density_each_row_copy = density_each_row
                        for u in range(self.n_copies - 1):
                            # density_each_row = np.kron(density_each_row, density_each_row_copy)
                            density_each_row = kronecker(density_each_row, density_each_row_copy)
                
                    # Calculate sum of quantum densities belonging to either class, per subset split
                    density_sum = density_sum + density_each_row
                return density_sum
            
            # Function to calculate centroid belonging to either class
            def centroid():
                # Calculate sum of quantum densities belonging to either class
                # density_sum_class = np.sum(Parallel(n_jobs = self.n_jobs) \
                                          # (delayed(X_prime_class_split_func)(j) for j in range(self.n_splits)), axis = 0)
                if __name__ == '__main__':
                    with Pool(processes = self.n_jobs) as p1:
                        density_sum_list = p1.map(X_prime_class_split_func, range(self.n_splits))
                        density_sum_class = torch.stack(density_sum_list, dim = 0).sum(dim = 0)
                
                # Calculate Quantum Centroid belonging to either class
                # Added ZeroDivisionError as required by sklearn check_estimator
                try:
                    centroid = (1 / m_class)*density_sum_class
                except ZeroDivisionError:
                    centroid = 0   #**************
                return centroid
            
            # Calculate centroid belonging to either class
            centroid_class = centroid()
                            
            # Calculate terms in the quantum Helstrom observable belonging to either class
            if self.class_wgt == 'equi':
                q_hels_obs_terms = 0.5*centroid_class
            elif self.class_wgt == 'weighted':
                q_hels_obs_terms = (m_class / m)*centroid_class
            else:
                raise ValueError('class_wgt should be "equi" or "weighted"')
            return m_class, centroid_class, q_hels_obs_terms
                            
        # Calculate Quantum Centroids and terms in the quantum Helstrom observable belonging to either class
        # centroid_terms = np.array(Parallel(n_jobs = self.n_jobs) \
                                          # (delayed(centroid_terms_func)(i) for i in range(2)))
        if __name__ == '__main__':
            with Pool(processes = self.n_jobs) as p2:
                centroid_terms = p2.map(centroid_terms_func, range(2))
              
        # Determine Quantum Centroids
        self.centroid_ = centroid_terms[:, 1]
           
        # Calculate quantum Helstrom observable
        self.q_hels_obs_ = centroid_terms[0, 2] - centroid_terms[1, 2]     
        
        # Calculate eigenvalues w and eigenvectors v of the quantum Helstrom observable
        # w, v = np.linalg.eigh(self.q_hels_obs_)
        w, v = torch.eig(self.q_hels_obs_, eigenvectors = True)
        
        # Length of w
        # len_w = len(w)
        len_w = len(w[:, 0])
        
        # Initialize array eigval_class
        # eigval_class = np.empty_like(w)
        eigval_class = torch.empty_like(w[:, 0], dtype = torch.float64).cuda()
        for i in range(len_w):
            # Create an array of 0s and 1s to indicate positive and negative eigenvalues
            # respectively
            # if w[i] > 0:
            if w[:, 0][i] > 0:
                eigval_class[i] = 0
            else:
                eigval_class[i] = 1
        
        # Transpose matrix v containing eigenvectors to row-wise
        eigvec = v.T
        
        # Function to calculate sum of the projectors corresponding to positive and negative
        # eigenvalues respectively
        def sum_proj_func(i):
            # Split eigenvectors belonging to positive or negative eigenvalues into n_splits subsets
            # eigvec_class_split = np.array_split(eigvec[eigval_class == i], 
                                                # indices_or_sections = self.n_splits, 
                                                # axis = 0)
            eigvec_class_split = torch.chunk(eigvec[eigval_class == i], 
                                             chunks = self.n_splits, 
                                             dim = 0)
            
            # Function to calculate sum of the projectors corresponding to positive and negative
            # eigenvalues respectively, per subset split
            def eigvec_class_split_func(j):
                # Initialize array proj_sum_split
                proj_sum_split = torch.zeros_like(self.q_hels_obs_, dtype = torch.float64).cuda()
                for k in eigvec_class_split[j]:
                    # Calculate sum of the projectors corresponding to positive and negative
                    # eigenvalues respectively, per subset split
                    # proj_sum_split = proj_sum_split + np.dot(k.reshape(-1, 1), k.reshape(1, -1))
                    proj_sum_split = proj_sum_split + torch.matmul(k.reshape(-1, 1), k.reshape(1, -1))
                return proj_sum_split        
            # return np.sum(Parallel(n_jobs = self.n_jobs) \
                         # (delayed(eigvec_class_split_func)(j) for j in range(self.n_splits)), axis = 0)
            if __name__ == '__main__':
                with Pool(processes = self.n_jobs) as p3:
                    proj_sum_split_list = p3.map(eigvec_class_split_func, range(self.n_splits))
            return torch.stack(proj_sum_split_list, dim = 0).sum(dim = 0)
        
        # Calculate sum of the projectors corresponding to positive and negative eigenvalues
        # respectively
        # self.proj_sum_ = Parallel(n_jobs = self.n_jobs) \
                         # (delayed(sum_proj_func)(i) for i in range(2))
        if __name__ == '__main__':
            with Pool(processes = self.n_jobs) as p4:
                self.proj_sum_ = p4.map(sum_proj_func, range(2))
                       
        # Calculate Helstrom bound
        # self.hels_bound_ = (centroid_terms[0, 0] / m)*np.einsum('ij,ji->', self.centroid_[0], 
                                                                # self.proj_sum_[0]) \
                           # + (centroid_terms[1, 0] / m)*np.einsum('ij,ji->', self.centroid_[1], 
                                                                  # self.proj_sum_[1])
        self.hels_bound_ = (centroid_terms[0, 0] / m)*torch.einsum('ij,ji->', self.centroid_[0], 
                                                                   self.proj_sum_[0]).item() \
                           + (centroid_terms[1, 0] / m)*torch.einsum('ij,ji->', self.centroid_[1], 
                                                                     self.proj_sum_[1]).item()
        return self
        
    
    # Function for predict_proba
    def predict_proba(self, X):
        """Performs HQC classification on X and returns the trace of the dot product of the densities 
        and the sum of the projectors with corresponding positive and negative eigenvalues respectively.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.       
            
        Returns
        -------
        trace_matrix : array-like, shape (n_samples, 2)
            Column index 0 corresponds to the trace of the dot product of the densities and the sum  
            of projectors with positive eigenvalues. Column index 1 corresponds to the trace of the  
            dot product of the densities and the sum of projectors with negative eigenvalues. An array 
            of float.
        """
        # Send tensor self.proj_sum_ from GPU to CPU and cast into an array
        self.proj_sum_arr_ = self.proj_sum_.cpu().numpy()
                
        # Check if fit had been called
        # check_is_fitted(self, ['proj_sum_'])
        check_is_fitted(self, ['proj_sum_arr_'])

        # Cast tensor X into an array
        X_arr = X.numpy()        
        
        # Input validation
        # X = check_array(X)
        X = check_array(X_arr)
        
        # Send tensor X from CPU to GPU
        X = X.cuda()     
        
        # Cast X to float to ensure all following calculations below are done in float 
        # rather than integer
        # X = X.astype(float)  
        X = torch.DoubleTensor(X)
        
        # Rescale X
        X = self.rescale*X        
        
        # Calculate sum of squares of each row (sample) in X
        # X_sq_sum = (X**2).sum(axis = 1)
        X_sq_sum = (X**2).sum(dim = 1)
        
        # Number of rows in X
        m = X.shape[0]
        
        # Number of columns in X
        n = X.shape[1]

        # Calculate X' using amplitude or inverse of the standard stereographic projection 
        # encoding method
        if self.encoding == 'amplit':
            # X_prime = normalize(np.concatenate((X, np.ones(m).reshape(-1, 1)), axis = 1))
            X_prime = nn.functional.normalize(torch.cat([X, torch.ones(m, dtype=torch.float64).reshape(-1, 1).cuda()],
                                                        dim = 1), p = 2, dim = 1)
        elif self.encoding == 'stereo':
            # X_prime = (1 / (X_sq_sum + 1)).reshape(-1, 1) \
                      # *(np.concatenate((2*X, (X_sq_sum - 1).reshape(-1, 1)), axis = 1))
            X_prime = (1 / (X_sq_sum + 1)).reshape(-1, 1)*(torch.cat((2*X, (X_sq_sum - 1).reshape(-1, 1)), dim = 1))
        else:
            raise ValueError('encoding should be "amplit" or "stereo"')
                       
        # Function to calculate trace values for each class
        def trace_func(i):
            # Split X' into n_splits subsets, row-wise
            # X_prime_split = np.array_split(X_prime, 
                                           # indices_or_sections = self.n_splits, 
                                           # axis = 0)
            X_prime_split = torch.chunk(X_prime,
                                        chunks = self.n_splits,
                                        dim = 0)
            
            # Function to calculate trace values for each class, per subset split
            def trace_split_func(j):
                # Counter for j-th split X'
                X_prime_split_jth = X_prime_split[j]
                
                # Number of rows (samples) in j-th split X'
                X_prime_split_m = X_prime_split_jth.shape[0]
                
                # Initialize array trace_class_split
                # trace_class_split = np.empty(X_prime_split_m)
                trace_class_split = torch.empty(X_prime_split_m, dtype = torch.float64).cuda()
                for k in range(X_prime_split_m):
                    # Encode vectors into quantum densities
                    X_prime_split_each_row = X_prime_split_jth[k, :]
                    # density_each_row = np.dot(X_prime_split_each_row.reshape(-1, 1), 
                                              # X_prime_split_each_row.reshape(1, -1))
                    density_each_row = torch.matmul(X_prime_split_each_row.reshape(-1, 1),
                                                    X_prime_split_each_row.reshape(1, -1))
                                                                  
                    # Calculate n-fold Kronecker tensor product
                    if self.n_copies == 1:     
                        density_each_row = density_each_row
                    else:
                        density_each_row_copy = density_each_row
                        for u in range(self.n_copies - 1):
                            # density_each_row = np.kron(density_each_row, density_each_row_copy)
                            density_each_row = kronecker(density_each_row, density_each_row_copy)
                        
                    # Calculate trace of the dot product of density of each row and sum of projectors 
                    # with corresponding positive and negative eigenvalues respectively    
                    # trace_class_split[k] = np.einsum('ij,ji->', density_each_row, self.proj_sum_[i])
                    trace_class_split[k] = torch.einsum('ij,ji->', density_each_row, self.proj_sum_[i]).item()
                return trace_class_split
            
            # Calculate trace values for each class, per subset split
            # trace_class = Parallel(n_jobs = self.n_jobs) \
                          # (delayed(trace_split_func)(j) for j in range(self.n_splits))
            if __name__ == '__main__':
                with Pool(processes = self.n_jobs) as p5:
                    trace_class = p5.map(trace_split_func, range(self.n_splits))
            # return np.concatenate(trace_class, axis = 0)
            return torch.cat(trace_class, dim = 0)
            
        # Calculate trace values for each class
        # trace_matrix = np.transpose(Parallel(n_jobs = self.n_jobs) \
                                   # (delayed(trace_func)(i) for i in range(2)))
        if __name__ == '__main__':
            with Pool(processes = self.n_jobs) as p6:
                trace_matrix = torch.stack(p6.map(trace_func, range(2)), dim = 1)
        return trace_matrix
        
    
    # Function for predict
    def predict(self, X):
        """Performs HQC classification on X and returns the binary classes.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples. An array of int or float.
            
        Returns
        -------
        self.classes_[predict_trace_index] : array-like, shape (n_samples,)
            The predicted binary classes. An array of str, int or float.
        """
        # Determine column index with the higher trace value in trace_matrix
        # If both columns have the same trace value, returns column index 0
        # predict_trace_index = np.argmax(self.predict_proba(X), axis = 1)
        predict_trace_index = torch.argmax(self.predict_proba(X), axis = 1)
        # Returns the predicted binary classes
        return self.classes_[predict_trace_index]

In [ ]:
from sklearn.datasets import make_classification
import torch

# Create dataset
X, y = make_classification(n_samples=2000, n_features=20, n_classes=2, random_state=0)

# Cast X and y numpy arrays into Pytorch tensors as the HQC estimator accepts Pytorch tensors (stored in CPU)
X_tsr = torch.DoubleTensor(X)   # Tensor with floating point elements
y_tsr = torch.LongTensor(y)   # Tensor with integer elements

# Fit and predict model
model = HQC(rescale=0.5, n_copies=4, n_jobs=1, n_splits=2).fit(X_tsr, y_tsr)
y_hat_tsr = model.predict(X_tsr)

# Send tensor y_hat_tsr from GPU to CPU, and cast to numpy array
y_hat = y_hat_tsr.cpu().numpy()

# Check F1 score and Helstrom bound values
from sklearn import metrics
print(metrics.f1_score(y, y_hat))
print(model.Hels_bound_)